In [29]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
import logging
from sqlalchemy import text
import time
from IPython.display import display, Markdown
from ingesting import Database

### Connecting to Database

In [30]:
# Connect to the 'inventory' database now
db = Database(database_name='inventory')  # Should now say "inventory"

✅ Connected to database: inventory


In [31]:
# Checking the tables present in the database
table_names = db.read_sql_query("SELECT name FROM sys.tables;")
table_names

,name
0,begin_inventory
1,end_inventory
2,purchases
3,purchase_prices
4,sales
5,vendor_invoice


In [32]:
for i in table_names.name:
    print(i)

begin_inventory
end_inventory
purchases
purchase_prices
sales
vendor_invoice


In [33]:
# Checking table and their shapes
for table in table_names.name:
    display(Markdown(f"# **{'-'*30} {table} {'-'*30}**"))

    # pd.read_sql_query or pd.read_sql returns dataframe that's why ['cnt'].values[0] used
    count_result = db.read_sql_query(f'SELECT COUNT(*) as cnt FROM {table}')['cnt'].values[0]
    
    display(db.read_sql_query(f'SELECT TOP 3 * FROM {table}'))
    display(Markdown(f'#### **Count of records: {count_result}**'))

# **------------------------------ begin_inventory ------------------------------**

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01


#### **Count of records: 206529**

# **------------------------------ end_inventory ------------------------------**

,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31


...

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


#### **Count of records: 12825363**

# **------------------------------ vendor_invoice ------------------------------**

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None


#### **Count of records: 5543**

In [34]:
# Now we will check fix vendor number in every table
purchases = db.read_sql_query('SELECT * FROM purchases WHERE VendorNumber = 4466')
purchases

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [35]:
purchase_prices = db.read_sql_query('SELECT * FROM purchase_prices WHERE VendorNumber = 4466')
purchase_prices

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE
1,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
2,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE


In [36]:
vendor_invoice = db.read_sql_query('SELECT * FROM vendor_invoice WHERE VendorNumber = 4466')
vendor_invoice 

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [37]:
sales = db.read_sql_query('SELECT * FROM sales WHERE VendorNo = 4466')
sales 

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9448,9_BLACKPOOL_5215,9,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-12-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9449,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-02,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9450,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9451,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [38]:
purchases.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

In [39]:
purchases.groupby(['Brand','PurchasePrice'])[['Quantity','Dollars']].sum()
# This table contains quantity and amount of objects purchased

,,Quantity,Dollars
Brand,PurchasePrice,,
3140,11.19,4640,51921.60
5215,9.41,4923,46325.43
5255,9.35,6215,58110.25


In [40]:
purchase_prices
# This table contains prices of objects which has purchased

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE
1,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
2,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE


In [41]:
sales.groupby('Brand')[['SalesDollars','SalesPrice','SalesQuantity']].sum()

,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
3140,50531.10,30071.85,3890
5215,60416.49,41542.02,4651
5255,79187.04,51180.60,6096


- The purchases table contains actual purchase data, including the date of purchase, products(brands), purchased by vendors, the amount paid (in dollars), and the quantity purchased.

- The purchase price column is derived from purchase_price table, which provides product-wise actual and purchase price. The combination of vendor and brand is unique in this table.

- The vendor_invoice table aggregates data from the purchase table, summarizing quantity and dollar amounts, along with an additional column for freight. This table maintains uniqueness based on vendor and PO number.

- The sales table captures actual sales transaction, detailing the brands purchased by vendors, the quantity sold, the selling price, and the revenue earned.

### As the data that we need for analysis is distributed in different tables, we need to create a summary table containing:

- purchase transaction made by vendors
- sales transaction data
- freight costs for each vendor 
- actual product prices from vendors

In [42]:
vendor_invoice.columns

Index(['VendorNumber', 'VendorName', 'InvoiceDate', 'PONumber', 'PODate',
       'PayDate', 'Quantity', 'Dollars', 'Freight', 'Approval'],
      dtype='object')

In [43]:
freight_summary = db.read_sql_query('SELECT VendorNumber, Sum(Freight) as FreightCost FROM vendor_invoice GROUP BY VendorNumber')
freight_summary

,VendorNumber,FreightCost
0,6830,360.29
1,8920,232.99
2,17032,283.61
3,7749,91.50
4,4692,9139.38
...,...,...
121,11567,20964.81
122,2450,251.28
123,7239,16978.67
124,8664,18544.49


In [44]:
db.read_sql_query("""SELECT 
                        p.VendorNumber,
                        p.VendorName,
                        p.Brand,
                        SUM(p.Quantity) as TotalPurchaseQuantity,
                        SUM(p.Dollars) as TotalPurchaseDollars   
                    FROM purchases as p
                    INNER JOIN purchase_prices as pp
                        ON p.Brand = pp.Brand
                    WHERE p.PurchasePrice > 0
                    GROUP BY p.VendorNumber, p.VendorName, p.Brand
                    ORDER BY TotalPurchaseDollars""")

,VendorNumber,VendorName,Brand,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,1,0.71
1,3960,DIAGEO NORTH AMERICA INC,6127,1,1.47
2,3924,HEAVEN HILL DISTILLERIES,9123,2,1.48
3,8004,SAZERAC CO INC,5683,6,2.34
4,9815,WINE GROUP INC,8527,2,2.64
...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,138109,3023206.01
10688,3960,DIAGEO NORTH AMERICA INC,4261,201682,3261197.94
10689,17035,PERNOD RICARD USA,8068,187407,3418303.68
10690,4425,MARTIGNETTI COMPANIES,3405,164038,3804041.22


In [45]:
sales.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [46]:
db.read_sql_query('''SELECT 
                        VendorNo,
                        Brand,
                        SUM(SalesDollars) as TotalSalesDollars,
                        SUM(SalesPrice) as TotalSalesPrice,
                        SUM(ExciseTax) as TotalExciseTax
                    FROM sales
                    GROUP BY VendorNo, Brand
                    ORDER BY TotalSalesDollars;''')

,VendorNo,Brand,TotalSalesDollars,TotalSalesPrice,TotalExciseTax
0,8004,5287,9.800000e-01,0.98,0.10
1,9206,2773,9.900000e-01,0.99,0.05
2,3252,3933,1.980000e+00,0.99,0.10
3,10050,3623,1.980000e+00,1.98,0.10
4,3924,9123,1.980000e+00,0.99,0.10
...,...,...,...,...,...
11267,3960,3545,4.223108e+06,545778.28,249587.83
11268,3960,4261,4.475973e+06,420050.01,368242.80
11269,17035,8068,4.538121e+06,461140.15,343854.07
11270,4425,3405,4.819073e+06,561512.37,294438.66


In [47]:
import time
start = time.time()

vendor_sales_summary_final = db.read_sql_query("""
WITH FreightSummary as (SELECT 
							   VendorNumber,
							   SUM(Freight) as FreightCost
						FROM Vendor_invoice
						GROUP BY VendorNumber
),

PurchaseSummary as (SELECT
						  p.VendorNumber,
						  p.VendorName,
						  p.Brand,
						  p.Description,
						  p.PurchasePrice,
						  pp.Price as ActualPrice,
						  pp.Volume,
						  SUM(p.Quantity) as TotalPurchaseQuantity,
						  SUM(p.Dollars) as TotalPurchaseDollars
					FROM purchases as p
					JOIN purchase_prices as pp
					     ON p.Brand = pp.Brand
					WHERE p.PurchasePrice > 0
					GROUP BY p.VendorNumber,p.VendorName,p.Brand,p.Description,p.PurchasePrice,pp.Price,pp.Volume
),

SalesSummary as (SELECT
						VendorNo,
						Brand,
						SUM(SalesQuantity) as TotalSalesQuantity,
						SUM(SalesPrice) as TotalSalesPrice,
						SUM(SalesDollars) as TotalSalesDollars,
						SUM(ExciseTax) as TotalExciseTax
				FROM sales
				GROUP BY VendorNo, Brand
)

SELECT
		ps.VendorNumber,
		ps.VendorName,
		ps.Brand,
		ps.Description,
		ps.PurchasePrice,
		ps.ActualPrice,
		ps.Volume,
		ps.TotalPurchaseQuantity,
		ps.TotalPurchaseDollars,
		ss. TotalSalesQuantity,
		ss. TotalSalesDollars,
		ss. TotalSalesPrice,
		ss.TotalExciseTax,
		fs.FreightCost
FROM PurchaseSummary as ps
LEFT JOIN SalesSummary as ss
ON ps. VendorNumber = ss.VendorNo
	AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary as fs
	ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC;""")

end = time.time()
print(f'Time taken {(end-start)/60} min')
vendor_sales_summary_final

Time taken 0.6072778701782227 min


,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750,145080,3811251.60,142049.0,5.101920e+06,672819.31,260999.20,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750,164038,3804041.22,160247.0,4.819073e+06,561512.37,294438.66,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750,187407,3418303.68,187140.0,4.538121e+06,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750,201682,3261197.94,200412.0,4.475973e+06,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750,138109,3023206.01,135838.0,4.223108e+06,545778.28,249587.83,257032.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750,2,2.64,5.0,1.595000e+01,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50,6,2.34,134.0,6.566000e+01,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50,2,1.48,2.0,1.980000e+00,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200,1,1.47,72.0,1.432800e+02,77.61,15.12,257032.07


In [48]:
vendor_sales_summary = vendor_sales_summary_final.copy()

### This query generates a vendor-wise sales and purchase summary, which is valuable for:

#### Performance Optimization:

- The query involves heavy joins and aggregations on large datasets like sales and purchases.
- Storing the pre-aggregated results avoids repeated expensive computations.
- Helps in analyzing sales, purchases, and pricing for different vendors and brands.
- Future Benefits of Storing this data for faster Dashboarding & Reporting.
- Instead of running expensive queries each time, dashboards can fetch data quickly from vendor_sales_summary.

In [49]:
vendor_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorNumber           10692 non-null  int64  
 1   VendorName             10692 non-null  object 
 2   Brand                  10692 non-null  int64  
 3   Description            10692 non-null  object 
 4   PurchasePrice          10692 non-null  float64
 5   ActualPrice            10692 non-null  float64
 6   Volume                 10692 non-null  object 
 7   TotalPurchaseQuantity  10692 non-null  int64  
 8   TotalPurchaseDollars   10692 non-null  float64
 9   TotalSalesQuantity     10514 non-null  float64
 10  TotalSalesDollars      10514 non-null  float64
 11  TotalSalesPrice        10514 non-null  float64
 12  TotalExciseTax         10514 non-null  float64
 13  FreightCost            10692 non-null  float64
dtypes: float64(8), int64(3), object(3)
memory usage: 1.1+ 

In [50]:
vendor_sales_summary.isnull().sum()

VendorNumber               0
VendorName                 0
Brand                      0
Description                0
PurchasePrice              0
ActualPrice                0
Volume                     0
TotalPurchaseQuantity      0
TotalPurchaseDollars       0
TotalSalesQuantity       178
TotalSalesDollars        178
TotalSalesPrice          178
TotalExciseTax           178
FreightCost                0
dtype: int64

In [51]:
vendor_sales_summary[vendor_sales_summary['TotalSalesPrice'].isnull()]

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
2456,9165,ULTRA BEVERAGE COMPANY LLP,1069,Spade&Bushel 10yr Irish Wsky,29.84,39.99,375,804,23991.36,NaN,NaN,NaN,NaN,68054.70
2528,9165,ULTRA BEVERAGE COMPANY LLP,1074,Brothership Whiskey,39.99,49.99,750,570,22794.30,NaN,NaN,NaN,NaN,68054.70
2982,9165,ULTRA BEVERAGE COMPANY LLP,3949,Patron En Lalique Tequila,5681.81,7499.99,750,3,17045.43,NaN,NaN,NaN,NaN,68054.70
3007,9165,ULTRA BEVERAGE COMPANY LLP,1071,Straw Boys Vodka,14.70,19.99,750,1140,16758.00,NaN,NaN,NaN,NaN,68054.70
3026,9552,M S WALKER INC,2256,Tamdhu Batch Speyside Whisky,72.46,99.99,750,228,16520.88,NaN,NaN,NaN,NaN,55551.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10628,9552,M S WALKER INC,24820,Ch Bourbon La Chapelle Medoc,10.13,14.99,750,1,10.13,NaN,NaN,NaN,NaN,55551.82
10634,5612,MILTONS DISTRIBUTING CO,23568,Criterion Rioja,9.72,14.49,750,1,9.72,NaN,NaN,NaN,NaN,9.46
10638,3960,DIAGEO NORTH AMERICA INC,4397,Capt Morgan Parrot Bay Mango,9.44,11.99,750,1,9.44,NaN,NaN,NaN,NaN,257032.07
10660,9815,WINE GROUP INC,13875,Concannon Selct Petite Sirah,6.84,9.99,750,1,6.84,NaN,NaN,NaN,NaN,27100.41


In [52]:
vendor_sales_summary['VendorName'].value_counts()

VendorName
MARTIGNETTI COMPANIES          1382
ULTRA BEVERAGE COMPANY LLP      837
M S WALKER INC                  834
PERFECTA WINES                  783
E & J GALLO WINERY              503
                               ... 
FANTASY FINE WINES CORP           1
AAPER ALCOHOL & CHEMICAL CO       1
SILVER MOUNTAIN CIDERS            1
CAPSTONE INTERNATIONAL            1
FLAVOR ESSENCE INC                1
Name: count, Length: 129, dtype: int64

In [53]:
vendor_sales_summary['VendorName'].str.strip().str.len()

0        17
1        21
2        17
3        24
4        24
         ..
10687    14
10688    14
10689    24
10690    24
10691    20
Name: VendorName, Length: 10692, dtype: int64

In [54]:
vendor_sales_summary[vendor_sales_summary['VendorName'].str.len() != vendor_sales_summary['VendorName'].str.strip().str.len()]
# There are some extra spaces

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750,145080,3811251.60,142049.0,5.101920e+06,672819.31,260999.20,68601.68
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750,187407,3418303.68,187140.0,4.538121e+06,461140.15,343854.07,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750,201682,3261197.94,200412.0,4.475973e+06,420050.01,368242.80,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750,138109,3023206.01,135838.0,4.223108e+06,545778.28,249587.83,257032.07
5,480,BACARDI USA INC,3858,Grey Goose Vodka,17.77,23.99,750,138809,2466635.93,141860.0,3.383912e+06,446932.09,111699.19,89286.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,4.99,750,2,2.64,5.0,1.595000e+01,10.96,0.55,27100.41
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,0.49,50,6,2.34,134.0,6.566000e+01,1.47,7.04,50293.62
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,0.99,50,2,1.48,2.0,1.980000e+00,0.99,0.10,14069.87
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,1.99,200,1,1.47,72.0,1.432800e+02,77.61,15.12,257032.07


In [55]:
vendor_sales_summary['Description'].value_counts()

Description
Southern Comfort                10
Bacardi Superior Rum             7
Jagermeister Liqueur             7
Capt Morgan Spiced Rum           6
Southern Comfort 100 Proof       6
                                ..
Tijuana White Tequilla           1
Ch Lestiac Cotes de Bordeaux     1
Redtree Moscato                  1
Promesses de France CdRhone      1
Firefly Caramel Moonshine        1
Name: count, Length: 9651, dtype: int64

In [56]:
vendor_sales_summary[vendor_sales_summary['Description'].str.len() != vendor_sales_summary['Description'].str.strip().str.len()]
# No extra spaces

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost


In [57]:
vendor_sales_summary['Volume'] = vendor_sales_summary['Volume'].astype(float)

vendor_sales_summary.fillna(0,inplace=True)

vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [58]:
# Making new column GrossProfit
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [59]:
vendor_sales_summary[vendor_sales_summary['GrossProfit']<1]
# Gross loss    

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit
304,3960,DIAGEO NORTH AMERICA INC,2244,Crown Royal Vanilla,17.90,23.99,750.0,10379,185784.10,7615.0,176577.85,85513.21,5996.56,257032.07,-9206.25
331,3960,DIAGEO NORTH AMERICA INC,3214,Ciroc Mango Vodka,19.99,24.99,750.0,8702,173952.98,6566.0,164084.34,29588.16,5170.69,257032.07,-9868.64
437,4425,MARTIGNETTI COMPANIES,44714,Buehler Znfdl Napa,13.51,19.99,750.0,10242,138369.42,9925.0,119616.75,47116.97,1108.44,144929.24,-18752.67
464,6213,MHW LTD,5929,RumChata Liqueur,34.88,44.99,1750.0,3767,131392.96,3224.0,130097.76,96065.23,5926.79,9621.58,-1295.20
550,3960,DIAGEO NORTH AMERICA INC,2468,Crown Royal Nrth Harvest Rye,22.39,29.99,750.0,5168,115711.52,3971.0,113966.29,83504.91,3130.07,257032.07,-1745.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10634,5612,MILTONS DISTRIBUTING CO,23568,Criterion Rioja,9.72,14.49,750.0,1,9.72,0.0,0.00,0.00,0.00,9.46,-9.72
10638,3960,DIAGEO NORTH AMERICA INC,4397,Capt Morgan Parrot Bay Mango,9.44,11.99,750.0,1,9.44,0.0,0.00,0.00,0.00,257032.07,-9.44
10660,9815,WINE GROUP INC,13875,Concannon Selct Petite Sirah,6.84,9.99,750.0,1,6.84,0.0,0.00,0.00,0.00,27100.41,-6.84
10672,5612,MILTONS DISTRIBUTING CO,17444,D'Aquino Asti,5.22,7.99,750.0,1,5.22,0.0,0.00,0.00,0.00,9.46,-5.22


In [60]:
# Calculating Profit Margin
def profit_margin(row):
    if row['TotalSalesDollars'] == 0:
        return 0
    else:
        return (row['GrossProfit'] / row['TotalSalesDollars']) * 100


vendor_sales_summary['ProfitMargin'] = vendor_sales_summary.apply(profit_margin,axis=1)

In [61]:
# Calculating Stock Turnover
vendor_sales_summary['StockTurnover'] = vendor_sales_summary['TotalSalesQuantity']/vendor_sales_summary['TotalPurchaseQuantity']

In [62]:
# Calculating Sales to Purchase ratio
vendor_sales_summary['SalestoPurchaseRatio'] = vendor_sales_summary['TotalSalesDollars']/vendor_sales_summary['TotalPurchaseDollars']

In [63]:
vendor_sales_summary.head()

,VendorNumber,VendorName,Brand,Description,PurchasePrice,ActualPrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,FreightCost,GrossProfit,ProfitMargin,StockTurnover,SalestoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,36.99,1750.0,145080,3811251.60,142049.0,5.101920e+06,672819.31,260999.20,68601.68,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,28.99,1750.0,164038,3804041.22,160247.0,4.819073e+06,561512.37,294438.66,144929.24,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,24.99,1750.0,187407,3418303.68,187140.0,4.538121e+06,461140.15,343854.07,123780.22,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,22.99,1750.0,201682,3261197.94,200412.0,4.475973e+06,420050.01,368242.80,257032.07,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,29.99,1750.0,138109,3023206.01,135838.0,4.223108e+06,545778.28,249587.83,257032.07,1199901.61,28.412764,0.983556,1.396897


In [64]:
db.close_connection()

🔒 Connection closed.


In [65]:
vendor_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   VendorNumber           10692 non-null  int64  
 1   VendorName             10692 non-null  object 
 2   Brand                  10692 non-null  int64  
 3   Description            10692 non-null  object 
 4   PurchasePrice          10692 non-null  float64
 5   ActualPrice            10692 non-null  float64
 6   Volume                 10692 non-null  float64
 7   TotalPurchaseQuantity  10692 non-null  int64  
 8   TotalPurchaseDollars   10692 non-null  float64
 9   TotalSalesQuantity     10692 non-null  float64
 10  TotalSalesDollars      10692 non-null  float64
 11  TotalSalesPrice        10692 non-null  float64
 12  TotalExciseTax         10692 non-null  float64
 13  FreightCost            10692 non-null  float64
 14  GrossProfit            10692 non-null  float64
 15  Pr

In [66]:
float_cols = vendor_sales_summary.select_dtypes(include=['float64','int']).columns

for col in float_cols:
    vendor_sales_summary[col] = vendor_sales_summary[col].round(2)
    print(f'Minimum value of {col} is : {vendor_sales_summary[col].min()}')
    print(f'Maximum value of {col} is : {vendor_sales_summary[col].max()}')
    print(50*'=')

Minimum value of VendorNumber is : 2
Maximum value of VendorNumber is : 201359
Minimum value of Brand is : 58
Maximum value of Brand is : 90631
Minimum value of PurchasePrice is : 0.36
Maximum value of PurchasePrice is : 5681.81
Minimum value of ActualPrice is : 0.49
Maximum value of ActualPrice is : 7499.99
Minimum value of Volume is : 50.0
Maximum value of Volume is : 20000.0
Minimum value of TotalPurchaseQuantity is : 1
Maximum value of TotalPurchaseQuantity is : 337660
Minimum value of TotalPurchaseDollars is : 0.71
Maximum value of TotalPurchaseDollars is : 3811251.6
Minimum value of TotalSalesQuantity is : 0.0
Maximum value of TotalSalesQuantity is : 334939.0
Minimum value of TotalSalesDollars is : 0.0
Maximum value of TotalSalesDollars is : 5101919.51
Minimum value of TotalSalesPrice is : 0.0
Maximum value of TotalSalesPrice is : 672819.31
Minimum value of TotalExciseTax is : 0.0
Maximum value of TotalExciseTax is : 368242.8
Minimum value of FreightCost is : 0.09
Maximum value o

In [67]:
vendor_sales_summary.to_csv('vendor_sales_summary.csv',index=False)

### Connecting to 'inventory' database

In [68]:
from ingesting import Database
db = Database(database_name='inventory')

✅ Connected to database: inventory


In [69]:
db.SQL_script_execution("""DROP TABLE IF EXISTS vendor_sales_summary""")

Current Database is : inventory
------------------------------
✅ Changes made successfully
🔒 Connection closed.


In [70]:
db.SQL_script_execution("""CREATE TABLE vendor_sales_summary (
                                    VendorNumber INT,
                                    VendorName VARCHAR(100),
                                    Brand INT,
                                    Description VARCHAR(100),
                                    PurchasePrice DECIMAL(10,2),
                                    ActualPrice DECIMAL(10,2),
                                    Volume INT,
                                    TotalPurchaseQuantity INT,
                                    TotalPurchaseDollars DECIMAL(15,2),
                                    TotalSalesQuantity INT,
                                    TotalSalesDollars DECIMAL(15,2),
                                    TotalSalesPrice DECIMAL(15,2),
                                    TotalExciseTax DECIMAL(15,2),
                                    FreightCost DECIMAL(15,2),
                                    GrossProfit DECIMAL(15,2),
                                    ProfitMargin DECIMAL(15,2),
                                    StockTurnover DECIMAL(15,2),
                                    SalesToPurchaseRatio DECIMAL(15,2),
                                    PRIMARY KEY (VendorNumber, Brand))""")

Current Database is : inventory
------------------------------
✅ Changes made successfully
🔒 Connection closed.


In [71]:
db.ingesting_data(df=vendor_sales_summary,table_name='vendor_sales_summary',if_exists='append')

Time taken in ingesting "vendor_sales_summary" is 0.01069021224975586 minutes
"vendor_sales_summary" is now ingested in "inventory" database
🔒 Connection closed.
